In [1]:
codigo = 'VALE3'

In [2]:
import pandas as pd
import requests
import json
from base64 import b64encode, b64decode

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# Função para buscar o tradingName pelo ticker
def get_trading_name(ticker):
    params = {"language": "pt-br", "pageNumber": 1, "pageSize": 20, "company": ticker}
    params = bytes(str(params), encoding="ascii")
    string = b64encode(params).decode()
    r = requests.get(
        'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetInitialCompanies/' +
        string,
        verify=False
    )
    for i in r.json()['results']:
        if i['issuingCompany'].lower() == ticker.lower():
            return i['tradingName'].replace('/', '').replace('.', '')
    raise ValueError('Empresa não encontrada')

# Função para obter os proventos
def proventos(ticker):
    tradingName = get_trading_name(ticker)
    results = []
    n = 1
    while True:
        params = {"language": "pt-br", "pageNumber": n, "pageSize": 100, "tradingName": tradingName}
        params = bytes(str(params), encoding="ascii")
        string = b64encode(params).decode()
        r = requests.get(
            'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/' +
            string,
            verify=False
        )
        if not r.json()['results']:
            break
        results += r.json()['results']
        n += 1
    return results


In [3]:
get_trading_name(codigo[:-1])

'VALE'

In [4]:
proventos_data = proventos(codigo[:-1])

In [5]:
data = pd.DataFrame(proventos_data)
data.head()

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx
0,ON,27/11/2025,"1,244102486",1,DIVIDENDO,11/12/2025,11/12/2025,"72,00",1,"1,727920",2025-12-11T00:00:00
1,ON,27/11/2025,"0,768133538",1,DIVIDENDO,11/12/2025,11/12/2025,"72,00",1,"1,066852",2025-12-11T00:00:00
2,ON,27/11/2025,"1,569535033",1,JRS CAP PROPRIO,11/12/2025,11/12/2025,"72,00",1,"2,179910",2025-12-11T00:00:00
3,ON,31/07/2025,"1,895387417",1,JRS CAP PROPRIO,12/08/2025,12/08/2025,"55,91",1,"3,390069",2025-08-12T00:00:00
4,ON,19/02/2025,"2,141847479",1,DIVIDENDO,07/03/2025,07/03/2025,"57,02",1,"3,756309",2025-03-07T00:00:00


In [6]:
data.tail()

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx
145,PNA,28/12/1999,"1,17",1,JRS CAP PROPRIO,28/12/1999,28/12/1999,"46,60",1,"2,510730",1999-12-28T00:00:00
146,PNA,04/08/1999,"1,11",1,JRS CAP PROPRIO,06/08/1999,06/08/1999,"34,50",1,"3,217391",1999-08-06T00:00:00
147,PNA,30/12/1998,"0,5",1,JRS CAP PROPRIO,30/12/1998,30/12/1998,"15,50",1,"3,225806",1998-12-30T00:00:00
148,PNA,28/10/1998,"0,65",1,JRS CAP PROPRIO,28/10/1998,28/10/1998,"18,30",1,"3,551913",1998-10-28T00:00:00
149,PNA,05/08/1998,"0,75",1,JRS CAP PROPRIO,05/08/1998,06/08/1998,"22,01",1,"3,407542",1998-08-05T00:00:00


In [7]:
import matplotlib.pyplot as plt

# Converte 'valueCash' de string com vírgula para float
data["valueCash"] = data["valueCash"].str.replace(",", ".").astype(float)
# Extrai o ano da data de aprovação
data["year"] = pd.to_datetime(data["dateApproval"], dayfirst=True).dt.year
# Agrupa por ano e soma os proventos
proventos_anuais = data.groupby("year")["valueCash"].sum()


In [8]:

# Gráfico com Plotly
fig = go.Figure(
    data=[
        go.Bar(
            x=data["year"],
            y=data["valueCash"],
            marker_color="royalblue",
            # text=data["valueCash"].round(3),
            # textposition="outside"
        )
    ]
)

fig.update_layout(
    title="Proventos Pagos por Ano",
    xaxis_title="Ano",
    yaxis_title="Total de Proventos (R$)",
    xaxis_tickangle=90,
    template="plotly_white",
    height=500
)

fig.show()


NameError: name 'go' is not defined

In [ ]:
# proventos via Fundamentus

from io import StringIO
import plotly.graph_objects as go

url1 = f'https://www.fundamentus.com.br/proventos.php?papel={codigo}&tipo=2'
header = { 
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36", "X-Requested-With": "XMLHttpRequest"
    }
r1 = requests.get(url1, headers=header)
dy = pd.read_html(StringIO(r1.text), decimal=',', thousands='.')[0]
dy['Data'] = pd.to_datetime(dy['Data'] , dayfirst=True)
dy.set_index('Data',inplace =True )

dy['Valor/acao'] = dy['Valor']/dy['Por quantas ações']

fig = go.Figure(
    [ go.Bar(x=dy.index.year , y = dy['Valor'],name='Valor',
            textposition="outside"),
     go.Bar(x=dy.index.year , y = dy['Valor/acao'],name='Valor/ação',
            textposition="outside")],
    layout=dict(title='Dividendos ',height=500, width=700 ))
fig.show()

Fonte: https://fabiomt.medium.com/como-obter-o-hist%C3%B3rico-de-dividendos-e-outros-proventos-com-python-direto-do-site-da-b3-90a507623418


testes:

In [ ]:
s = 'eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjIwLCJ0cmFkaW5nTmFtZSI6IkJSQVNJTCJ9'
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/'
r=requests.get(url + s)
r.json()

{'page': {'pageNumber': 1,
  'pageSize': 20,
  'totalRecords': 268,
  'totalPages': 14},
 'results': [{'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,04583263233',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '02/12/2025',
   'dateClosingPricePriorExDate': '02/12/2025',
   'closingPricePriorExDate': '22,56',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,203159',
   'lastDateTimePriorEx': '2025-12-02T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,07192713139',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '01/12/2025',
   'dateClosingPricePriorExDate': '01/12/2025',
   'closingPricePriorExDate': '22,26',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,323123',
   'lastDateTimePriorEx': '2025-12-01T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '14/05/2025',
   'valueCash': '0,33425840109',
   'ratio': '1',
   'corporateAction': 'JR

In [ ]:
padrao = b64decode(s)
padrao

b'{"language":"pt-br","pageNumber":1,"pageSize":20,"tradingName":"BRASIL"}'

In [ ]:
# max item/pagina = 120
s = b64encode(b'{"language":"pt-br","pageNumber":1,"pageSize":120,"tradingName":"BRASIL"}').decode()
s

'eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjEyMCwidHJhZGluZ05hbWUiOiJCUkFTSUwifQ=='

In [ ]:
r=requests.get(url + s)
print(r.json()['results'][-1])
r.json()

{'typeStock': 'ON', 'dateApproval': '11/08/2015', 'valueCash': '0,00032874943', 'ratio': '1', 'corporateAction': 'RENDIMENTO', 'lastDatePriorEx': '21/08/2015', 'dateClosingPricePriorExDate': '21/08/2015', 'closingPricePriorExDate': '18,56', 'quotedPerShares': '1', 'corporateActionPrice': '0,001771', 'lastDateTimePriorEx': '2015-08-21T00:00:00'}


{'page': {'pageNumber': 1,
  'pageSize': 120,
  'totalRecords': 268,
  'totalPages': 3},
 'results': [{'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,04583263233',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '02/12/2025',
   'dateClosingPricePriorExDate': '02/12/2025',
   'closingPricePriorExDate': '22,56',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,203159',
   'lastDateTimePriorEx': '2025-12-02T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '10/11/2025',
   'valueCash': '0,07192713139',
   'ratio': '1',
   'corporateAction': 'JRS CAP PROPRIO',
   'lastDatePriorEx': '01/12/2025',
   'dateClosingPricePriorExDate': '01/12/2025',
   'closingPricePriorExDate': '22,26',
   'quotedPerShares': '1',
   'corporateActionPrice': '0,323123',
   'lastDateTimePriorEx': '2025-12-01T00:00:00'},
  {'typeStock': 'ON',
   'dateApproval': '14/05/2025',
   'valueCash': '0,33425840109',
   'ratio': '1',
   'corporateAction': 'JR